## A basic ML exercise

We will load necessary packages first. Makesure you are using tensorflow kernel. 

In [ ]:
import tensorflow as tf
import pandas as pd
import keras
from keras.utils import np_utils, multi_gpu_model
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Activation, Dropout, add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, ModelCheckpoint

import ROOT
from sys import exit
import numpy as np

import csv
from sklearn.utils import shuffle
import os

If GPU is available, we can use GPU with following line. 

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Read ntuple within RDataFrame.

In [ ]:
df_signal = ROOT.RDataFrame("mytree", "myntuple_ttH.root")
df_background = ROOT.RDataFrame("mytree", "myntuple_ttbb.root")

Change root data format directly to numpy within RDataFrame. Then will check the format of the numpy. 

In [ ]:
df_train_p = df_signal.AsNumpy()
df_train_n = df_background.AsNumpy()

In [ ]:
print("Read-out of the full RDataFrame:\n{}\n".format(df_train_p))
print("Read-out of the full RDataFrame:\n{}\n".format(df_train_n))

It has a keyword and corresponding values for each event as columns. So we will stack all variables that will be used as input for deep neural network. 

In [ ]:
train_p = np.vstack( ( df_train_p["dR"], df_train_p["mbb"]  ))
train_n = np.vstack( ( df_train_n["dR"], df_train_n["mbb"]  ))
print( train_p.shape )
print(train_p)

Then, will need to change the shape so that column represents variables and row represents each event. Compare the shape of the matrix. (Please let me know if you have a better idea to simply this procedure.)

And check number of events for signal and background

In [ ]:
train_p = np.transpose(train_p)
train_n = np.transpose(train_n)
print( train_p.shape )
print(train_p)
print("signal events = ",  len(train_p))
print("background events = ",  len(train_n))

We will label the signal as 1 and background as 0. 

In [ ]:
train_data = np.vstack(( train_p, train_n))
train_label = np.array([1]*len(train_p)+[0]*len(train_n))

Shuffling the events. 

In [ ]:
numbertr=len(train_label)
#Shuffling
order=shuffle(range(numbertr),random_state=100)
train_label=train_label[order]
train_data=train_data[order,0::]
train_label = train_label.reshape( (numbertr, 1) )

Splitting between training set and cross-validation set. In this exercise, 90% of data will be used for training and 10% for test. 

In [ ]:
trainnb=0.9 # Fraction used for training
valid_data=train_data[int(trainnb*numbertr):numbertr,0::]
valid_label=train_label[int(trainnb*numbertr):numbertr]

train_data=train_data[0:int(trainnb*numbertr),0::]
train_label=train_label[0:int(trainnb*numbertr)]

Check the input variables. Here in this exercise, we have only two variables: `dR` and `mbb`. This one you can add more variables by modifying the [ana.C](https://github.com/monttj/CMSDAS2019/blob/master/ana.C) macro. 

In [ ]:
nvar = train_data.shape[1]
print("number of input variables = " , nvar)

Initial parameter setting

In [ ]:
a = 50 # number of nodes
b = 0.08 # dropout probability
#init
init = 'glorot_uniform'#called "Xavier uniform initializer"

We will build model now and save it to `model.h5`. Adam optimizer is used in this exercise. 

In [ ]:
inputs = Input(shape=(nvar,))
x = Dense(a, activation='relu', kernel_initializer=init, bias_initializer='zeros')(inputs)
x = Dropout(b)(x)
x = Dense(a, activation='relu', kernel_initializer=init, bias_initializer='zeros')(x)
x = Dropout(b)(x)
x = Dense(a, activation='relu', kernel_initializer=init, bias_initializer='zeros')(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)

adam=keras.optimizers.Adam(lr=1E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1E-3)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_accuracy'])

modelfile = 'model.h5'

Start training with 20 epochs with batch size of 1024. 

In [ ]:
checkpoint = ModelCheckpoint(modelfile, monitor='val_binary_accuracy', verbose=1, save_best_only=False)

history = model.fit(train_data, train_label,
                             epochs=20, batch_size=1024,
                             validation_data=(valid_data,valid_label),
                             )

Will check the performance by looking at the loss as a function of epochs.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Binary crossentropy')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Test'],loc='upper right')
plt.savefig(os.path.join("./",'fig_score_loss.pdf'))

plt.show()

We can also plot the accuracy as a function of epochs. 

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Test'], loc='lower right')
plt.savefig(os.path.join("./",'fig_score_acc.pdf'))

plt.show()

### Task

Can you lower the loss and increase the accuracy by changing the number of layers or optimize the parameters or adding more input features?